# Segmentazione di edifici da immagini satellitari e classificazione del contesto urbano: notebook per training del modello di riconoscimento

*Progetto di Visione Artificiale e Riconoscimento 2024/2025*

- **Nome:** Pablo Sebastian
- **Cognome:** Vargas Grateron
- **Email:** pablo.vargasgrateron@studio.unibo.it

Questo progetto si concentra sulla segmentazione di edifici da immagini satellitari e sulla classificazione del contesto urbano. Utilizza tecniche di deep learning per analizzare immagini aeree e identificare edifici e altre strutture urbane.

Il dataset utilizzato è il dataset [Inria Aerial Image Labeling](https://project.inria.fr/aerialimagelabeling/), che contiene immagini aeree ad alta risoluzione e le relative maschere di segmentazione degli edifici.

> Assicurati di scaricare il dataset prima di eseguire il notebook e di avere al meno 50 GB di spazio libero su disco.

## Librerie e percorsi dei dataset

Il codice contenuto nelle seguenti celle importa le librerie necessarie e definisce i percorsi per i dataset utilizzati nel progetto.

> Assicurati di avere installato le librerie richieste contenute nel file `requirements.txt` prima di eseguire il notebook.

In [ ]:
import os, time, random
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
print("TF version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TF version: 2.19.0
Num GPUs Available:  1


Per la prossima cella, assicurati di avere i seguenti percorsi impostati correttamente:
- `PATH_DATASET_DIR`: percorso alla cartella principale dove sono memorizzati i dataset.
- `PATH_DATASET`: percorso alla cartella contenente i dataset originali.
- `PATH_PROCESSED_DATASET`: percorso alla cartella dove saranno memorizzati i dataset elaborati.
- `PATH_MODEL`: percorso alla cartella dove sarà memorizzato il modello addestrato.

In [2]:
PATH_DATASET_DIR = Path("../../dataset")
PATH_DATASET = PATH_DATASET_DIR / "original"
PATH_PROCESSED_DATASET = PATH_DATASET_DIR / "processed"

PATH_TRAIN_DATASET = PATH_DATASET / "train"
PATH_TEST_DATASET = PATH_DATASET / "test"

PATH_TRAIN_DATASET_PROCESSED = PATH_PROCESSED_DATASET / "train"
PATH_TEST_DATASET_PROCESSED = PATH_PROCESSED_DATASET / "test"

PATH_MODEL = Path("model")

## Setup: caricamento del modello e processing delle immagini

Per utilizzare il modello addestrato, è necessario caricarlo e preparare le immagini per la classificazione. Nella seguente cella, eseguiamo queste operazioni.

> Per procedere, assicurati di avere il modello addestrato nel primo notebook `1_model_training.ipynb`.

In [ ]:
JACCARD_INDEX_SMOOTH = 1e-12

def jaccard_index(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
    jac = (intersection + JACCARD_INDEX_SMOOTH) / (sum_ - intersection + JACCARD_INDEX_SMOOTH)
    return K.mean(jac)

In [10]:
dependencies = {'jaccard_index': jaccard_index}
loaded_model=keras.models.load_model(PATH_MODEL / "UnetWSatellite.keras", custom_objects=dependencies)